In [ ]:
!pip install SpeechRecognition pydub ffmpy converter ffmpeg-python audioread google-cloud-speech --upgrade

In [1]:
import requests
import speech_recognition as sr
import os
import shutil
import ffmpy
import subprocess
import glob
  
from pydub import AudioSegment 
from pydub.silence import split_on_silence
from pydub.playback import play

In [ ]:
folder = "discursos"
os.chdir(folder) 

files = glob.glob("*.mp4")
for filename in files:
    filenameMp3 = filename[:-4]
    print("Traspasando mp4 a wav el archivo: " + filename)
    os.system('ffmpeg -i {} -vn -acodec pcm_s16le -ar 44100 -ac 2 {}.wav'.format(filename, filenameMp3))

    print("moviendo "+ filename +" a archivos pasados a wav")
    origen = filename
    destino = "Mp4Transformados/"+filename
    shutil.move(origen, destino)

os.chdir('..')

In [4]:
os.chdir('discursos')
files = glob.glob("*.wav")
os.chdir('..')

r = sr.Recognizer()
print("inicia")
for file in files:
    print("Inicia archivo: "+file)

    silence_based_conversion("discursos/",file)
    audio2text(file)

    print("moviendo archivo")
    origen = "discursos/"+file
    destino = "discursos/WavListos/"+file
    shutil.move(origen, destino)

    shutil.rmtree('audio_chunks')
print("termina")


inicia
eru-wodh-sxv (2020-07-09 at 05_48 GMT-7).wav
discursos/eru-wodh-sxv (2020-07-09 at 05_48 GMT-7).wav
cargando wav
['discursos', '.gitignore', 'SpeechToTextLocal.ipynb', 'creaCarpetas.sh', 'audio2texto.py', 'trnasformarAmano.txt', 'README.md', '.git', 'SpeechToTextLocalColab.ipynb']
split del audio por silencios


KeyboardInterrupt: 

In [2]:
def silence_based_conversion(path,archivo): 
  
    # open the audio file stored in 
    # the local system as a wav file.
    print(path+archivo)
    print("cargando wav")
    print(os.listdir())
    song = AudioSegment.from_wav(path+archivo)
    #play(song)
          
    # split track where silence is 0.5 seconds  
    # or more and get chunks 
    print("split del audio por silencios")
    chunks = split_on_silence(song, 
        # must be silent for at least 0.5 seconds 
        # or 500 ms. adjust this value based on user 
        # requirement. if the speaker stays silent for  
        # longer, increase this value. else, decrease it. 
        min_silence_len = 500, 
        # consider it silent if quieter than -16 dBFS 
        # adjust this per requirement 
        silence_thresh = song.dBFS-16,
        keep_silence=500
    ) 
  
    # create a directory to store the audio chunks. 
    try: 
        print("creacion del fichero audio_chunks")
        os.mkdir('audio_chunks') 
    except(FileExistsError): 
        pass
  
    # move into the directory to 
    # store the audio files. 
    print("entra al fichero audio_chunks")
    os.chdir('audio_chunks') 
  
    i = 0
    # process each chunk 
    print("se inicia el prceso de los audios")
    for chunk in chunks: 
              
        # Create 0.5 seconds silence chunk 
        chunk_silent = AudioSegment.silent(duration = 100) 
  
        # add 0.5 sec silence to beginning and  
        # end of audio chunk. This is done so that 
        # it doesn't seem abruptly sliced. 
        audio_chunk = chunk_silent + chunk + chunk_silent 
  
        # export audio chunk and save it in  
        # the current directory. 
        print("saving chunk{0}.wav".format(i)) 
        # specify the bitrate to be 192 k 
        audio_chunk.export("./chunk{0}.wav".format(i), bitrate ='192k', format ="wav") 
  
        # the name of the newly created chunk 
        filename = 'chunk'+str(i)+'.wav'
        print("Processing chunk "+str(i)+" - filename: "+filename) 
        
        i += 1
  
        

    print("paso atras en la carpeta")
    os.chdir('..') 
  

In [3]:
def audio2text(archivo = "recognized"):
    
    os.chdir("audio_chunks")

    # open a file where we will concatenate   
    # and store the recognized text 
    print("creando archivo recognized.txt")
    fh = open("../discursos/"+archivo[:-4]+".txt", "w+") 

    audios = os.listdir()

    i = 0
    for audio in audios:
        # get the name of the newly created chunk 
        # in the AUDIO_FILE variable for later use.
        file = "chunk{0}.wav".format(i)
        i += 1
        print(file)

        # create a speech recognition object 
        r = sr.Recognizer()

        # recognize the chunk 
        with sr.AudioFile(file) as source: 

            #song = AudioSegment.from_wav(audio)
            #play(song)

            # remove this if it is not working 
            # correctly. 
            print("cargando Archivo")
            #r.adjust_for_ambient_noise(source) 
            audio_listened = r.listen(source)

        try: 
            print("------------- tratando de convertir en texto -------------")
            # try converting it to text 
            rec = r.recognize_google(audio_listened,language="es-ES") 
            print(rec)
            # write the output to the file. 
            fh.write(rec+". ") 
        
        # catch any errors. 
        except sr.UnknownValueError: 
            print("Could not understand audio") 

        except sr.RequestError as e: 
            print("Could not request results. check your internet connection")
    
    os.chdir('..')